<a href="https://colab.research.google.com/github/wangbxj1234/xformerforpt/blob/main/Linearformer%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn


class linear_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes):
        super(linear_Attention, self).__init__()
        self.n_heads =8
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo,k) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           S=tgt_len
           Q = self.query_projection(x_s).view(L, self.n_heads, -1)
           K = self.key_projection(x_s).view(S, self.n_heads, -1)
           V = self.value_projection(x_s).view(S, self.n_heads, -1)
           #Q = queries.transpose(0, 1)
           #K = keys.transpose(0, 1)
           #V = values.transpose(0, 1)
           # 2. Non-negative projection
           Q = self.kernel_method(Q)
           K = self.kernel_method(K)

           KV = torch.einsum("shd,shm->hmd", K, V)

           # Compute the normalizer
           Z = 1/(torch.einsum("lhd,hd->lh", Q, K.sum(dim=0))+self.eps)

           # Finally compute and return the new values
           V = torch.einsum("lhd,hmd,lh->lhm", Q, KV, Z)

           output = V.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out


In [5]:
o=[ 838, 2088, 2317, 3567, 4541, 5561, 6427, 7077]
o=torch.IntTensor(o)
x = torch.rand(7077, 128)
p = torch.rand(7077, 3)

In [6]:
planes=128
transformer2 = linear_Attention(planes)
right_res = transformer2([p, x, o],k=16)
print(right_res.shape)

torch.Size([7077, 128])
